# multiprocessing

> [Main Table of Contents](../../README.md)

## In This Notebook
- Sharing data betwee processes
- `start` method - deeper look
    - Change start method

In [ ]:
# Start at Query process daemon

## [Multiprocessing in python comprehensive guide](https://superfastpython.com/multiprocessing-in-python/)

The main process is the instance of the python interpreter that is used to run a python program. Therefore, any process created within a python program using the multiprocessing package will be child processes of the main parent process that is running the top-level python program.  Child processes can create their own child processes.

- An instance of the multiprocessing.Process class provides a handle of a new instance of the Python interpreter
- Each process has a name and can be changed (google later on how to do so)
    - The parent process has the name “MainProcess“
    - Child processes are named automatically in a somewhat unique manner within each process with the form “Process-%d” where %d is the integer indicating the process number created by the parent process, e.g. Process-1 for the first process created
    
        ```python
        # report the process name
        print(process.name)
        ```

Create a separate process (technically a child process in the main python program).
1) Create a new process
2) Start a process (start as soon as possible (doesn't necessarily start immediately) the main thread within the process with a new instance of python interpreter).  `<process_name>.start()`
Note `.start()` method returns immediately, so it schedules to start the separate child process as soon as possible and returns control back the the main parent process
`.start()` internally calls `.run()`
3) [Optional] If I want to wait for the new process created in step 1 to finish executing, this can be done by `.join()` method.  This will block the main process, go back to the new process and wait until that process closed/terminated.

## Sharing data between processes
> Any changes made in one process is always propagated and made available to other processes
- multiprocessing.Value
- multiprocessing.Array

In [14]:
from multiprocessing import Value, Process

# Create a shared counter, initialize integer attribute
counter = Value('i', 0)

# Increment the counter in a child process
def increment_counter():
    ct = 0
    while ct < 10:
        print(f'child process counter: {counter.value}')
        counter.value += 1
        ct += 1
    

# Create a child process and start the increment_counter function
p = Process(target=increment_counter)
p.start()

# Check the value of the counter
print(f'parent process counter: {counter.value}')

parent process counter: 0
child process counter: 0
child process counter: 1

In [15]:
from multiprocessing import Value, Process

# Create a shared counter, initialize integer attribute
counter = Value('i', 0)

# Increment the counter in a child process
def increment_counter():
    ct = 0
    while ct < 10:
        print(f'child process counter: {counter.value}')
        counter.value += 1
        ct += 1
    

# Create a child process and start the increment_counter function
p = Process(target=increment_counter)
p.start()
p.join()  # blocks parent process (this process) until child process (p) is done

# Check the value of the counter
print(f'parent process coutner: {counter.value}')


child process counter: 2
child process counter: 3
child process counter: 4
child process counter: 5
child process counter: 6
child process counter: 7
child process counter: 8
child process counter: 9


child process counter: 0
child process counter: 1
child process counter: 2
child process counter: 3
child process counter: 4
child process counter: 5
child process counter: 6
child process counter: 7
child process counter: 8
child process counter: 9
parent process coutner: 10


## start method - deeper look
- `.start()` method is the technique used to start child processes in Python
- It can be one of three types:
    - spawn: start a new Python process
    - fork: copy a Python process from an existing process
    - forkserver: new process from which future forked processes will be copied
- Default start type depends on the platform
    - Windows (win32): spawn
    - macOS (darwin): spawn
    - Linux (unix): fork
- Note not all start method types are supported by all platforms
    - `multiprocessing.get_all_start_methods()` to get list of supported start methods
    - `multiprocessing.get_start_method()` to get the type of the current start method
    - Windows (win32): spawn
    - macOS (darwin): spawn, fork, forkserver.
    - Linux (unix): spawn, fork, forkserver

### Change the start method
- `multiprocessing.set_start_method(<start_method_type>)` e.g. `multiprocessing.set_start_method('spawn')`
- Required on most platforms that the start method be set first, prior to any other code, and to be done so within a if __name__ == ‘__main__’ check 
    ```python
    # protect the entry point
    if __name__ == '__main__':
        # set the start method
        multiprocessing.set_start_method('spawn')
    ```